# Add Cubes Around Surface of Precentral Gyrus

In [1]:
from freesurfer_surface import Surface

SUBJECTS_DIR = '../tests/subjects'
surface = Surface.read_triangular(SUBJECTS_DIR + '/fabian/surf/lh.pial')
surface.load_annotation_file(SUBJECTS_DIR + '/fabian/label/lh.aparc.annot')

precentral_label, = filter(lambda l: l.name == 'precentral', surface.annotation.labels.values())
precentral_label

Label(name=precentral, index=24, color=#3c14dc)

In [2]:
border_chain, = surface.find_label_border_polygonal_chains(precentral_label)
border_chain.normalized()

PolygonalChain(vertex_indices=(32065, 32072, 32073, 32080, 33464, 33471, 33472, 33479, 33486, 34919, 34920, 33497, 34932, 34946, 36359, 37820, 37832, 39293, 39306, 40864, 42390, 43954, 43967, 45559, 47010, 48397, 48417, 49856, 51276, 52723, 52724, 52744, 52745, 52746, 54085, 54086, 54087, 54088, 54089, 54090, 54108, 54109, 54110, 52778, 52779, 51323, 51324, 51325, 51326, 51327, 51343, 51344, 52796, 52797, 52798, 52799, 54150, 55369, 56555, 57747, 58890, 58889, 58903, 60001, 60013, 61045, 61044, 62119, 62118, 62107, 62118, 63255, 63264, 64542, 64562, 65878, 65895, 65925, 65957, 65958, 67222, 67255, 68439, 68458, 69743, 69744, 71109, 72350, 73534, 74843, 76105, 76087, 77297, 77310, 78461, 78462, 79548, 79560, 80703, 81917, 83081, 83082, 84257, 85504, 85505, 86741, 88079, 89444, 90726, 90727, 91981, 91982, 93119, 93120, 93121, 93122, 94229, 94230, 94237, 94238, 94239, 94240, 94241, 94251, 94252, 94265, 95347, 95348, 95337, 96419, 97510, 98721, 98741, 99968, 99979, 99993, 98779, 98778, 987

In [3]:
import numpy
from freesurfer_surface import Vertex, Triangle

def unit_vector(vector: numpy.array) -> numpy.array:
    return vector / numpy.linalg.norm(vector)

BORDER_WIDTH = 0.8 # mm

for vertex_triplet_indices in border_chain.adjacent_vertex_indices(3):
    vertex_triplet = surface.select_vertices(vertex_triplet_indices)
    backward_vector = vertex_triplet[0] - vertex_triplet[1]
    forward_vector = unit_vector(vertex_triplet[2] - vertex_triplet[1]) * BORDER_WIDTH / 2
    upward_vector = unit_vector(numpy.cross(backward_vector, forward_vector)) * BORDER_WIDTH / 2
    sideward_vector = unit_vector(numpy.cross(upward_vector, forward_vector)) * BORDER_WIDTH / 2
    cross_section_corners = [vertex_triplet[1] + v
                             for v in [-upward_vector -sideward_vector,
                                       -upward_vector +sideward_vector,
                                       +upward_vector +sideward_vector,
                                       +upward_vector -sideward_vector]]
    corner_indices = [[surface.add_vertex(Vertex(*(coords + forward_dir))) for coords in cross_section_corners]
                      for forward_dir in [-forward_vector, +forward_vector]]
    for surface_corner_indices in corner_indices:
        surface.add_rectangle(surface_corner_indices)
    for i in range(4):
        j = (i + 1) % 4
        surface.add_rectangle((corner_indices[0][i], corner_indices[0][j],
                               corner_indices[1][j], corner_indices[1][i]))

surface.write_triangular('precentral-border.lh.pial')

```sh
freeview --surface examples/precentral-border.lh.pial:annot=tests/subjects/fabian/label/lh.aparc.annot
```